In [1]:
import pandas as pd
import os
import re

In [2]:
def converttojson(df, dfname):# Convert DataFrame to JSON
    json_data = df.to_json(orient='records')
    file_path = f"Clean Data/{dfname}.json"

    # Optionally, save to a file if needed
    with open(file_path, 'w') as f:
        f.write(json_data)
    
    print(f"DataFrame '{dfname}' saved as JSON file: {file_path}")

def firstrowtocolname(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    column_names = df.iloc[0]
    # Remove the first row from the DataFrame
    df = df[1:]
    # Set the extracted row as column names
    df.columns = column_names
    # Reset the index after removing the first row
    df.reset_index(drop=True, inplace=True)
    return df

def fingertipsclean(df):
    df= df[['AreaName','Time period', 'Value']]
    df['AreaName'] = df['AreaName'].replace('London region', 'London')
    df_filtered = df[-6:] #take the last six rows (2 years of data)
    return df_filtered

def censusclean(df):
    new_columns=[df.columns[0]]
    for i, col in enumerate(df.columns):
        if pd.isna(col):
            new_col_name = f"{df.columns[i-1]} (%)"
            new_columns.append(new_col_name)
        elif i != 0:
            new_col_name = f"{df.columns[i]} (number)"
            new_columns.append(new_col_name)
    df.columns = new_columns
    df = df.iloc[1:]
    return df
def crimerate(rawdf, offencegroup='', subgroup=''):
    rawdf = rawdf.rename(columns={'Area name':'AreaName', 'Area Name':'AreaName'})
    rawdf['AreaName'] = rawdf['AreaName'].str.replace(' and ', ' & ')
    df = rawdf[rawdf['AreaName'].isin(wards+['Lambeth']) & 
            (rawdf['Measure']=='Offences')]
    if offencegroup != '':
        df = df[(df['Offence Group'] == offencegroup)]
    if subgroup != '':
        df= df[df['Offence Subgroup'] == subgroup]
    
    df = df.iloc[:,[0,3,10]]
    column_mapping = {df.columns[0]: 'Time period'}
    df = df.rename(columns=column_mapping)
    df['Count'] = pd.to_numeric(df['Count'], errors='coerce')
    df = df.groupby(['Time period','AreaName'], as_index=False).sum()

    londondf = rawdf[(rawdf['Area Type']=='Borough') & (~rawdf['AreaName'].str.contains('Aviation')) & 
            (rawdf['Measure']=='Offences')]
    if offencegroup != '':
        londondf = londondf[(londondf['Offence Group'] == offencegroup)]
    if subgroup!= '':
        londondf=londondf[londondf['Offence Subgroup'] == subgroup]
    londondf = londondf.iloc[:,[0,3,10]]
    column_mapping = {londondf.columns[0]: 'Time period'}
    londondf = londondf.rename(columns=column_mapping)
    londondf['Count'] = pd.to_numeric(londondf['Count'], errors='coerce')
    londondf = londondf.groupby(['Time period'], as_index=False).sum()
    londondf['AreaName'] = 'London'
    combineddf = pd.concat([df, londondf], ignore_index=True)
    combineddf = pd.merge(pop, combineddf, on='AreaName', how='inner')
    combineddf['Time period'] = combineddf['Time period'].str.replace(r'-01$', '', regex=True)    
    combineddf['Population'] = pd.to_numeric(combineddf['Population'], errors='coerce')
    combineddf['Offences per 1,000 people'] = (combineddf['Count']/combineddf['Population'] * 1000).round(1)
    combineddf=combineddf.drop(columns=['Female','Male'])
    return combineddf

def create_surv_q_df(df, start_text="test col x vs y and z", end_text_select=1):
    surv_sat = df.iloc[:, :37]
    indices_to_drop = list(range(3, 12))
    surv_sat = surv_sat.drop(surv_sat.columns[indices_to_drop], axis=1)
    question = str(surv_sat.iloc[0,0])

    names = surv_sat.iloc[3, 2:].values
    # Create boolean masks with no NaN values
    start_mask = surv_sat.iloc[:,0].str.contains(start_text, regex=False, na=False)
    end_mask = surv_sat.iloc[:,0].str.contains("Don’t know|Prefer not to say|NET", regex=True, na=False)

    # Find start and end indices for filtering
    if not start_mask.any() or not end_mask.any():
        raise ValueError("Start or end text not found in 'Response' column.")

    start_index = surv_sat[start_mask].index[0] + 1
    end_indices = surv_sat[end_mask].index
    if len(end_indices) < end_text_select:
        raise ValueError("Insufficient matches for end text to use the specified end_text_select value.")
    end_index = end_indices[end_text_select - 1] - 1

    surv_sat_f = surv_sat.loc[start_index:end_index]

    columns=['Response','Percentage of respondents']
    columns.extend(names)

    # Rename the rest of the columns to 'Percentage of respondents'
    column_mapping = {old_name: new_name for old_name, new_name in zip(surv_sat_f.columns, columns)}
    surv_sat_f = surv_sat_f.rename(columns=column_mapping)
    surv_sat_f = surv_sat_f.dropna(subset=[surv_sat_f.columns[0]])

    # Replace numbers in square brackets in the 'Response' column
    surv_sat_f['Response'] = surv_sat_f['Response'].str.replace(r"\[\d+\] ", "", regex=True)

    # Convert 'Percentage of respondents' to numeric and handle any non-numeric gracefully
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    # Multiply 'Percentage of respondents' by 100 to get percentage and round to 1 decimal place
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:] * 100
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].round(1)

    # Make column 'Question' with values from the question variable
    surv_sat_f['Question'] = question

    # Various string replacements and clean-ups on 'Question'
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace("Q:", "")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^Q[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^_[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.strip()
    return surv_sat_f

def create_surv_q_df2022(df, start_text="test col x vs y and z", end_text_select=1):
    surv_sat = df.iloc[:, :2]

    question = str(surv_sat.iloc[0,0])

    # Create boolean masks with no NaN values
    start_mask = surv_sat.iloc[:,0].str.contains(start_text, regex=False, na=False)
    end_mask = surv_sat.iloc[:,0].str.contains("Don’t know|Prefer not to say|NET", regex=True, na=False)

    # Find start and end indices for filtering
    if not start_mask.any() or not end_mask.any():
        raise ValueError("Start or end text not found in 'Response' column.")

    start_index = surv_sat[start_mask].index[0] + 1
    end_indices = surv_sat[end_mask].index
    if len(end_indices) < end_text_select:
        raise ValueError("Insufficient matches for end text to use the specified end_text_select value.")
    end_index = end_indices[end_text_select - 1] - 1

    surv_sat_f = surv_sat.loc[start_index:end_index]

    columns=['Response','Percent']

    # Rename the rest of the columns to 'Percentage of respondents'
    column_mapping = {old_name: new_name for old_name, new_name in zip(surv_sat_f.columns, columns)}
    surv_sat_f = surv_sat_f.rename(columns=column_mapping)
    surv_sat_f = surv_sat_f.dropna(subset=[surv_sat_f.columns[0]])

    # Replace numbers in square brackets in the 'Response' column
    surv_sat_f['Response'] = surv_sat_f['Response'].str.replace(r"\[\d+\] ", "", regex=True)

    
    # Convert 'Percentage of respondents' to numeric and handle any non-numeric gracefully
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    # Multiply 'Percentage of respondents' by 100 to get percentage and round to 1 decimal place
    surv_sat_f.iloc[:, 1] = surv_sat_f.iloc[:, 1] * 100
    surv_sat_f.iloc[:, 1] = surv_sat_f.iloc[:, 1].round(1)

    # Make column 'Question' with values from the question variable
    surv_sat_f['Question'] = question

    # Various string replacements and clean-ups on 'Question'
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace("Q:", "")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^Q[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^_[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.strip()
    return surv_sat_f
def trafficclean(traffic, timeperiod):
    traffic['_Casualty Count'] = traffic['_Casualty Count'].astype(int)

    # Assuming 'traffic' is your DataFrame containing traffic data
    # Aggregate Casualty Count by Borough Name
    traffic_b = traffic.groupby('Borough Name')['_Casualty Count'].sum().reset_index()

    # Rename 'Borough Name' to 'Area name'
    traffic_b = traffic_b.rename(columns={'Borough Name': 'AreaName'})

    # Filter to Lambeth
    traffic_lam = traffic_b[traffic_b['AreaName'] == 'Lambeth']

    # Filter to London
    traffic_lon = traffic_b.copy()
    traffic_lon['AreaName'] = 'London'
    traffic_lon = traffic_lon.groupby('AreaName')['_Casualty Count'].sum().reset_index()

    # Combine traffic_lam and traffic_lon
    traffic_b_out = pd.concat([traffic_lam, traffic_lon])

    # Merge population data to traffic_b
    traffic_b_out_m = pd.merge(traffic_b_out, pop, left_on='AreaName', right_on='AreaName', how='left')

    # Calculate rate
    traffic_b_out_m['Rate'] = ((traffic_b_out_m['_Casualty Count'] / traffic_b_out_m['Population']) * 1000).astype(float).round(1)

    # Rename columns
    traffic_b_out_m = traffic_b_out_m.rename(columns={
        '_Casualty Count': 'Number of traffic incidents casualties',
        'Rate': 'Number of traffic incidents casualties per 1,000 people'
    })

    traffic_b_out_m = traffic_b_out_m.iloc[:,[0,1,4,5]]
    traffic_b_out_m['Time period'] = timeperiod
    return traffic_b_out_m

def solarpanelclean(df, column, area):
    renew = df[df[column].str.contains(area)]

    # Convert photovoltaics and Estimated number of households to numeric
    renew[['Photovoltaics', 'Estimated number of households']] = renew[['Photovoltaics', 'Estimated number of households']].apply(pd.to_numeric)

    # Sum up all numeric columns by Region
    renew = renew.groupby(column)[['Photovoltaics', 'Estimated number of households']].sum().reset_index()

    # Create column percentage of households with solar PV
    renew['Number of solar PV installations per 1k households'] = (renew['Photovoltaics'] / (renew['Estimated number of households'] / 1000)).astype(float).round(2)

    # Rename column Region to Area name
    renew=renew.rename(columns={column: 'AreaName'})
    return renew

def load_pol(no2_d):
    # Rename columns
    no2_d.columns = ["AreaName", "Population 2019", 'Population projection 2025', 'Population projection 2030',
                     "Exceeding population 2019", "Exceeding population 2025", 'Exceeding population 2030', 
                     "Exceeding 2019 (%)", "Exceeding 2025 (%)", "Exceeding 2030 (%)"]

    # Convert Exceeding 2016 (%) and Exceeding 2019 (%) to numeric
    no2_d["Exceeding 2019 (%)"] = pd.to_numeric(no2_d["Exceeding 2019 (%)"], errors="coerce") * 100
    no2_d["Exceeding 2025 (%)"] = pd.to_numeric(no2_d["Exceeding 2025 (%)"], errors="coerce") * 100
    no2_d["Exceeding 2030 (%)"] = pd.to_numeric(no2_d["Exceeding 2030 (%)"], errors="coerce") * 100

    # Round Exceeding 2016 (%) and Exceeding 2019 (%) to two decimal places
    no2_d["Exceeding 2019 (%)"] = no2_d["Exceeding 2019 (%)"].round(2)
    no2_d["Exceeding 2025 (%)"] = no2_d["Exceeding 2025 (%)"].round(2)
    no2_d["Exceeding 2030 (%)"] = no2_d["Exceeding 2030 (%)"].round(2)

    # Keep only Lambeth, Central, Inner, Outer
    no2_d_out = no2_d[no2_d["AreaName"].isin(["Lambeth", "Central", "Inner", "Outer"])]
    no2_d_out = no2_d_out[['AreaName',"Exceeding 2019 (%)", "Exceeding 2025 (%)", "Exceeding 2030 (%)"]]

    # Rename Area name
    no2_d_out["AreaName"] = no2_d_out["AreaName"].replace({"Inner": "Inner London", 
                                                             "Outer": "Outer London", 
                                                             "Central": "Central London"})

    # Make Area name an ordered factor
    no2_d_out["AreaName"] = pd.Categorical(no2_d_out["AreaName"], 
                                            categories=["Lambeth", "Central London", "Inner London", "Outer London"], 
                                            ordered=True)

    # Melt DataFrame
    no2_d_out_m = pd.melt(no2_d_out, id_vars=["AreaName"], var_name="Time period", value_name="Exceeding (%)")

    # Replace "Exceeding" in Year column
    no2_d_out_m["Time period"] = no2_d_out_m["Time period"].str.replace(r' \(%\)$', "")
    no2_d_out_m["Time period"] = no2_d_out_m["Time period"].str.replace("Exceeding ", "")
    return no2_d_out_m

def industryclean(df, year):
    df = df.iloc[:3,:]
    df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
    melteddf= pd.melt(df, id_vars='Area', var_name='Industry', value_name='Value')

    # Split 'Industry' column into two categories
    melteddf[['Industry category no', 'Industry category']] = melteddf['Industry'].str.split(': ', expand=True)

    melteddf.columns = ['AreaName', 'Industry','Value','Industry category no', 'Industry category']

    # Calculate sums for each area
    total_values = melteddf.groupby('AreaName')['Value'].sum().astype(int).reset_index()
    total_values.columns = ['AreaName', 'Total Values']

    # Merge total values back into the original dataframe
    bus_df = melteddf.merge(total_values, on='AreaName')

    # Calculate percentages
    bus_df['Percentage'] = ((bus_df['Value'] / bus_df['Total Values']) * 100).astype(float).round(1)

    # Filter industries with significant presence in Lambeth
    bus_df_lambeth = bus_df[bus_df['AreaName'] == 'Lambeth']
    significant_industries = bus_df_lambeth[bus_df_lambeth['Percentage'] >= 2]['Industry']

    # Filter the original dataframe to keep only significant industries
    bus_df_filtered = bus_df[bus_df['Industry'].isin(significant_industries)]

    # Remove 'activities' from 'Industry' column
    bus_df_filtered['Industry category'] = bus_df_filtered['Industry category'].str.replace(" activities", "", regex=False)

    # Keep only relevant columns
    bus_df_filtered = bus_df_filtered[['Industry category', 'AreaName', 'Percentage']]

    # Order 'Industry' and 'Area' columns
    bus_df_filtered['Industry category'] = pd.Categorical(bus_df_filtered['Industry category'], categories=bus_df_filtered['Industry category'].unique(), ordered=True)
    bus_df_filtered['AreaName'] = pd.Categorical(bus_df_filtered['AreaName'], categories=['Lambeth', 'London', 'England'], ordered=True)
    bus_df_filtered['Time period'] = year
    return bus_df_filtered

## Concentration of NO2 and PM2.5 in the air
def conc_func(no2_conc_d):
    
    # Rename columns
    no2_conc_d.columns = ["AreaName", "Concentration 2013", "Concentration 2016", "Concentration 2019",
                          "Reduction 19-16 (ug/m3)", "Reduction 19-16 (%)", 'Concentration projection 2025', "Reduction 25-19 (ug/m3)", "Reduction 25-19 (%)"]
    
    # Convert columns to numeric
    numeric_cols = ["Concentration 2013", "Concentration 2016", "Concentration 2019",
                          "Reduction 19-16 (ug/m3)", "Reduction 19-16 (%)",'Concentration projection 2025',  "Reduction 25-19 (ug/m3)", "Reduction 25-19 (%)"]
    no2_conc_d[numeric_cols] = no2_conc_d[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Round numeric columns to one decimal place
    no2_conc_d[numeric_cols] = no2_conc_d[numeric_cols].round(1)
    
    no2_conc_d = no2_conc_d[no2_conc_d["AreaName"].isin(["Lambeth", "Inner", "Outer"])]
    no2_conc_d = no2_conc_d[['AreaName',"Concentration 2016", "Concentration 2019",
                          "Reduction 19-16 (ug/m3)", "Reduction 19-16 (%)",
                            'Concentration projection 2025',  "Reduction 25-19 (ug/m3)", "Reduction 25-19 (%)"]]

    # Replace values in 'Area name' column
    no2_conc_d['AreaName'] = no2_conc_d['AreaName'].replace({"Inner": "Inner London", "Outer": "Outer London"})
    
    # Convert 'Area name' to ordered categorical variable
    no2_conc_d['AreaName'] = pd.Categorical(no2_conc_d['AreaName'], categories=["Lambeth", "Inner London", "Outer London"], ordered=True)
    
    # Melt DataFrame
    no2_conc_d_out_m = pd.melt(no2_conc_d, id_vars="AreaName", var_name="Time period", value_name="Concentration")
    
    # Remove "Concentration " from 'Year' values
    no2_conc_d_out_m['Time period'] = no2_conc_d_out_m['Time period'].str.replace("Concentration ", "")
    
    return no2_conc_d_out_m

In [3]:
wards = ["Brixton Acre Lane","Brixton North","Brixton Rush Common","Brixton Windrush",
         "Clapham Common & Abbeville","Clapham East","Clapham Park","Clapham Town","Gipsy Hill",
         "Herne Hill & Loughborough Junction","Kennington","Knight's Hill","Myatt's Fields","Oval",
         "St Martin's","Stockwell East","Stockwell West & Larkhall","Streatham Common & Vale",
         "Streatham Hill East","Streatham Hill West & Thornton","Streatham St Leonard's",
         "Streatham Wells","Vauxhall","Waterloo & South Bank","West Dulwich"]

wardcodes= ['E05014095N', 'E05014096N','E05014097N','E05014098N','E05014099','E05014100','E05014101',
'E05014102','E05014103','E05014104','E05014105','E05014106','E05014107','E05014108','E05014109',
'E05014110','E05014111','E05014112','E05014113','E05014114','E05014115','E05014116','E05014117',
'E05014118','E05014119']

warddict = dict(zip(wardcodes, wards))

In [5]:
# Read the Excel file containing file names of datasets
data_sources = pd.read_excel('Data sources ambitions.xlsx')

## Import current data

In [6]:
dataframes = {}
# Iterate over the rows to get the file paths
for index, row in data_sources.iterrows():
    if not pd.isna(row['exported?']):
        continue
    #     if row['exported?'] == 'y': # comment out when re-importing everything
#         continue
    if pd.isna(row['File Name']):
        continue
    if row['Ambition'] == 'Golden Thread':
        ambition_folder = '..\Data\Golden Thread\\'
    else:
        ambition_folder = '..\Data\Ambition '+ str(row['Ambition']) + '\\' + str(row['Section'])
    
    file_name = str(row['File Name'])
    variable_name = row['Internal link suffix']
    sheet_name = str(row['Sheet']) if 'Sheet' in row and pd.notna(row['Sheet']) else 0
    skip = int(row['Skip rows']) if 'Skip rows' in row and pd.notna(row['Skip rows']) else None
    numrows = int(row['nrows']) if 'nrows' in row and pd.notna(row['nrows']) else None
    file_path = os.path.join(ambition_folder, file_name)
    print(file_path)

    # Load the data (example for CSV)
    if file_path.endswith('.csv'):
        # Try reading the CSV file with different encodings
        encodings_to_try = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252', 'utf-16']
        for encoding in encodings_to_try:
            try:
                dataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)
                print(f"CSV file successfully read with encoding: {encoding}")
                break  # Stop trying encodings if successful
            except UnicodeDecodeError:
                print(f"Failed to read CSV file with encoding: {encoding}") 
        
        # Perform operations on the data
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        dataframes[variable_name] = pd.read_excel(file_path, sheet_name = sheet_name, header=None, 
                                                  skiprows=skip, nrows=numrows)
    elif file_path.endswith('.ods'):
        dataframes[variable_name] = pd.read_excel(file_path, engine = 'odf', sheet_name = sheet_name, header=None, skiprows=skip, nrows=numrows)
        # Perform operations on the data
    if row['File Name'] == '2023 ARS crosstabs.xlsx':
        dataframes[variable_name] = create_surv_q_df(dataframes[variable_name])

    elif 'OtherCrime' in row['File Name']:
        dataframes[variable_name] = firstrowtocolname(dataframes[variable_name])
        column_mapping = {'Crime Type': 'Offence Group', 'Crime Subtype':'Offence Subgroup', 'Area Name' : 'AreaName'}
        dataframes[variable_name].rename(columns=column_mapping, inplace= True)
    else:
        dataframes[variable_name] = firstrowtocolname(dataframes[variable_name])
# uncomment when running everything again
    
    if row['Source organisation'] == 'Fingertips':
        dataframes[variable_name]= fingertipsclean(dataframes[variable_name])
        converttojson(dataframes[variable_name], variable_name)
    elif row['Source organisation'] == 'ONS':
        dataframes[variable_name] = dataframes[variable_name][:-3] #remove last three rows
        dataframes[variable_name] = dataframes[variable_name].dropna(how='all')
    elif row['Source organisation'] == 'ONS Census':
        # Remove anything before ":" in row names
        dataframes[variable_name].iloc[:,0] = dataframes[variable_name].iloc[:,0].str.split(':').str[-1]
        dataframes[variable_name] = dataframes[variable_name][:-2] #remove last two rows
        dataframes[variable_name] = dataframes[variable_name].dropna(how='all')
        dataframes[variable_name].columns = dataframes[variable_name].columns.str.replace(r'^.*:', '')
        dataframes[variable_name].columns = dataframes[variable_name].columns.str.replace(r'\s*\(Lambeth\)', '', regex=True)
        dataframes[variable_name].rename(columns={'Area': 'AreaName'}, inplace = True)
        if 'AreaName' in dataframes[variable_name].columns:
            dataframes[variable_name]['AreaName'] = dataframes[variable_name]['AreaName'].str.replace(r'\s*\(Lambeth\)', '', regex=True)
        if dataframes[variable_name].iloc[0].str.contains('number').any():
            dataframes[variable_name]= censusclean(dataframes[variable_name])
        converttojson(dataframes[variable_name], variable_name)

..\Data\Ambition 3\Key demographics\Identified Capacity 10-year migration.xlsx
..\Data\Ambition 3\Key demographics\sapewardstablefinal.xlsx


In [7]:
dataframes.keys()

dict_keys(['population-projection-London', 'population-by-age-and-sex'])

In [23]:
agesexproj = dataframes['population-by-age-and-sex']
agesexprojfiltered= agesexproj[agesexproj['Ward 2023 Name'].isin(wards)]
agesexprojfiltered= agesexprojfiltered[agesexprojfiltered['LAD 2023 Name']=='Lambeth'].iloc[:,3:]
# agesexprojfiltered
melted = pd.melt(agesexprojfiltered, id_vars = 'Ward 2023 Name', var_name='Age', value_name='Population')
total -
melted = melted[melted['Age'] != 'Total']


melted['Sex'] = melted['Age'].str[0]
melted['Sex'] = melted['Sex'].map({'M': 'Male', 'F': 'Female'})
melted['Age'] = melted['Age'].str[1:] #if melted['Age'][0].isin(['M','F']) else melted['Age']
print(melted)


                  Ward 2023 Name Age Population     Sex
25             Brixton Acre Lane   0         49  Female
26                 Brixton North   0         46  Female
27           Brixton Rush Common   0         82  Female
28              Brixton Windrush   0         51  Female
29    Clapham Common & Abbeville   0         52  Female
...                          ...  ..        ...     ...
4570      Streatham St Leonard's  90         22    Male
4571             Streatham Wells  90         23    Male
4572                    Vauxhall  90         16    Male
4573       Waterloo & South Bank  90         13    Male
4574                West Dulwich  90         18    Male

[4550 rows x 4 columns]


In [77]:
pop = dataframes['population-by-sex']
pop['Population'] = pop['Female'] +pop['Male']

In [215]:
popage = dataframes['population-by-age']
over65 = popage[['AreaName','Aged 65 to 74 years (number)','Aged 75 to 84 years (number)','Aged 85 years and over (number)']]
over65['Aged over 65 (2020)'] = over65.iloc[:,1:].sum(axis=1).astype('int')
over65= over65.iloc[:,[0,-1]]
over65

C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3288219667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over65['Aged over 65 (2020)'] = over65.iloc[:,1:].sum(axis=1).astype('int')


,AreaName,Aged over 65 (2020)
1,Lambeth,27265
2,Brixton Acre Lane,1141
3,Brixton North,1225
4,Brixton Rush Common,1107
5,Brixton Windrush,624
6,Clapham Common & Abbeville,774
7,Clapham East,699
8,Clapham Park,917
9,Clapham Town,1336
10,Gipsy Hill,955


## Import last year data (if available)

In [81]:
lydataframes = {}
# Iterate over the rows to get the file paths
for index, row in data_sources.iterrows():
    if not pd.isna(row['exported?']):
        continue
    #     if row['exported?'] == 'y': # comment out when re-importing everything
#         continue
    if pd.isna(row['Previous year File Name']) or row['File Name'] == row['Previous year File Name']:
        continue
    if row['Ambition'] == 'Golden Thread':
        ambition_folder = '..\Data\Golden Thread\\'
    else:
        ambition_folder = '..\Data\Ambition '+ str(row['Ambition']) + '\\' + str(row['Section'])
    
    file_name = str(row['Previous year File Name'])
    variable_name = 'lastyear' + row['Internal link suffix']
    if 'Previous year sheet' in row and pd.notna(row['Previous year sheet']):
        sheet_name = str(row['Previous year sheet'])
    else:
        sheet_name = str(row['Sheet']) if 'Sheet' in row and pd.notna(row['Sheet']) else 0
    skip = int(row['Skip rows']) if 'Skip rows' in row and pd.notna(row['Skip rows']) else None
    numrows = int(row['nrows']) if 'nrows' in row and pd.notna(row['nrows']) else None
    file_path = os.path.join(ambition_folder, file_name)
    print(file_path)
    
    # Load the data (example for CSV)
    if file_path.endswith('.csv'):
        # Try reading the CSV file with different encodings
        encodings_to_try = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252', 'utf-16']
        for encoding in encodings_to_try:
            try:
                lydataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)
                print(f"CSV file successfully read with encoding: {encoding}")
                break  # Stop trying encodings if successful
            except UnicodeDecodeError:
                print(f"Failed to read CSV file with encoding: {encoding}") 
        
        # Perform operations on the data
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        lydataframes[variable_name] = pd.read_excel(file_path, sheet_name = sheet_name, header=None, 
                                                  skiprows=skip, nrows=numrows)
    elif file_path.endswith('.ods'):
        lydataframes[variable_name] = pd.read_excel(file_path, engine = 'odf', sheet_name = sheet_name, header=None, skiprows=skip, nrows=numrows)
        # Perform operations on the data
    if row['Previous year File Name'] == 'res_survey_2022_filt.xlsx':
        lydataframes[variable_name] = create_surv_q_df2022(lydataframes[variable_name])
    elif 'OtherCrime' in row['Previous year File Name']:
        lydataframes[variable_name] = firstrowtocolname(lydataframes[variable_name])
        column_mapping = {'Crime Type': 'Offence Group', 'Crime Subtype':'Offence Subgroup', 'Area Name' : 'AreaName'}
        lydataframes[variable_name] = lydataframes[variable_name].rename(columns=column_mapping)
    else:
        lydataframes[variable_name] = firstrowtocolname(lydataframes[variable_name])

# uncomment when running everything again
#     if row['Source organisation'] == 'Fingertips':
#         dataframes[variable_name]= fingertipsclean(dataframes[variable_name])
#         converttojson(dataframes[variable_name], variable_name)
#     elif row['Source organisation'] == 'ONS':
#         lydataframes[variable_name] = lydataframes[variable_name][:-3] #remove last three rows
#         lydataframes[variable_name] = lydataframes[variable_name].dropna(how='all')

..\Data\Ambition 3\Jobs, earnings, and businesses\Pop simple 16-64 2021-22.xlsx
..\Data\Ambition 3\Jobs, earnings, and businesses\local units type 2023 NOMIS.xlsx


C:\Users\JKim\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\JKim\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [82]:
lydataframes.keys()

dict_keys(['lastyearworking-age-population21', 'lastyeartypes-of-industry22'])

# Wrangling

In [9]:
dataframes['types-of-industry'] #, 'qualifications', 'weekly-earnings-by-sex', 'earning-a-living-wage', 'unemployment-by-sex-and-ethnicity', 'unemployment-benefit-claimants', 'distance-to-work']

,Industry,lacu:Lambeth,country:England,gor:London
0,"01 : Crop and animal production, hunting and r...",10,90260,640
1,02 : Forestry and logging,5,3420,230
2,03 : Fishing and aquaculture,0,1725,45
3,05 : Mining of coal and lignite,0,10,0
4,06 : Extraction of crude petroleum and natural...,0,100,40
...,...,...,...,...
87,99 : Activities of extraterritorial organisati...,0,5,5
88,Column Total,14960,2737105,584415
89,NaN,NaN,NaN,NaN
90,All figures are rounded to avoid disclosure. V...,NaN,NaN,NaN


# Completed

In [98]:
#Type of Industry__________________________________________________________________
df = industryclean(dataframes['types-of-industry23'], 2023)
lydf = industryclean(lydataframes['lastyeartypes-of-industry22'], 2022)

combined_df = pd.concat([df, lydf], ignore_index=True)

converttojson(combined_df,'types-of-industry')
combined_df

DataFrame 'types-of-industry' saved as JSON file: Clean Data/types-of-industry.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_10020\2281606437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
C:\Users\JKim\AppData\Local\Temp\ipykernel_10020\2281606437.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_df_filtered['Industry category'] = bus_df_filtered['Industry category'].str.replace(" activities", "", regex=False)
C:\Users\JKim\AppData\Local\Temp\ipykernel_10020\2281606437.py:3: SettingWithCopyWarning: 
A value is trying to be s

,Industry category,AreaName,Percentage,Time period
0,Construction of residential and non-residentia...,Lambeth,2.1,2023
1,Retail sale in non-specialised stores,Lambeth,2.3,2023
2,Retail sale of other goods in specialised stores,Lambeth,2.5,2023
3,Restaurants and mobile food service,Lambeth,5.5,2023
4,"Motion picture, video and television programme",Lambeth,3.9,2023
...,...,...,...,...
79,Architectural and engineering and related tech...,London,2.2,2022
80,Specialised design,London,1.4,2022
81,Business support service n.e.c.,London,4.3,2022
82,"Creative, arts and entertainment",London,2.2,2022


In [80]:
# Anti-social behaviour =============================================================
asb=dataframes['anti-social-behaviour']
asb= asb[~asb['Safer_Neighborhood_Team_Borough_Name'].str.contains('Aviation', na=False)]
asbfiltered= asb[asb['Duplicate']=='No']
# #Check % NA
# (asbfiltered['Safer_Neighborhood_Team_Borough_Name'].isna().sum() / len(asbfiltered['Safer_Neighborhood_Team_Borough_Name']))
asbfiltered.dropna(subset=['Safer_Neighborhood_Team_Borough_Name'], inplace=True)
asbfiltered= asbfiltered[['ASBCount', 'Ward_WardCode','Safer_Neighborhood_Team_Name',
                         'Safer_Neighborhood_Team_Borough_Name']]
# asb.columns
asbfiltered.columns = ['Count', 'Wardcode', 'Ward','Borough']
asbfiltered['Count'] = asbfiltered['Count'].astype(int)
lonasb = asbfiltered.groupby(['Borough'])['Count'].sum().reset_index()
sum_count = lonasb['Count'].sum()

# Create a new DataFrame with the 'London' and sum_count as a single row
new_row = pd.DataFrame({'Borough': ['London'], 'Count': [sum_count]})

# Append the new row to your original DataFrame
lonasb = lonasb.append(new_row, ignore_index=True)
lonasb = lonasb[lonasb['Borough'].isin(['Lambeth','London'])]
lonasb.columns=['AreaName','Count']

wardsasb = asbfiltered[asbfiltered['Borough']=='Lambeth']
# wardsasb['Ward']=wardsasb['Ward'].replace(' and ', ' & ')
wardsasb = wardsasb[wardsasb['Ward'].isin(wards)]
wardsasb = wardsasb.groupby(['Ward'])['Count'].sum().reset_index()
wardsasb.columns = ['AreaName','Count']
combined_asb = pd.concat([lonasb, wardsasb], ignore_index=True)
combined_asb['Time period'] = 'May 2023-Apr 2024'

combineddf = pd.merge(pop, combined_asb, on='AreaName', how='inner')
combineddf['Population'] = pd.to_numeric(combineddf['Population'], errors='coerce')
combineddf['Offences per 1,000 people'] = (combineddf['Count']/combineddf['Population'] * 1000).round(1)
converttojson(combineddf, 'anti-social-behaviour')
combineddf

C:\Users\JKim\AppData\Local\Temp\ipykernel_9172\3897535188.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asbfiltered.dropna(subset=['Safer_Neighborhood_Team_Borough_Name'], inplace=True)
C:\Users\JKim\AppData\Local\Temp\ipykernel_9172\3897535188.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lonasb = lonasb.append(new_row, ignore_index=True)


DataFrame 'anti-social-behaviour' saved as JSON file: Clean Data/anti-social-behaviour.json


,AreaName,Female,Male,Population,Count,Time period,"Offences per 1,000 people"
0,Lambeth,163713,153941,317654,9092,May 2023-Apr 2024,28.6
1,Brixton Acre Lane,7517,7735,15252,371,May 2023-Apr 2024,24.3
2,Brixton North,8443,7881,16324,432,May 2023-Apr 2024,26.5
3,Brixton Rush Common,7895,7035,14930,355,May 2023-Apr 2024,23.8
4,Brixton Windrush,4920,4562,9482,198,May 2023-Apr 2024,20.9
5,Clapham Common & Abbeville,5192,4823,10015,182,May 2023-Apr 2024,18.2
6,Clapham East,5198,4647,9845,299,May 2023-Apr 2024,30.4
7,Clapham Park,6603,6108,12711,230,May 2023-Apr 2024,18.1
8,Clapham Town,8127,7777,15904,331,May 2023-Apr 2024,20.8
9,Gipsy Hill,5473,4784,10257,295,May 2023-Apr 2024,28.8


In [21]:
# Pollutant concentrations ===========================================

# % population exposed to >10 micrograms/m3
dfpm=dataframes['pollutant-concentrations-PM25'].iloc[:,14:24]
dfno2=dataframes['pollutant-concentrations-NO2'].iloc[:,14:24]
no2_d_out_m = load_pol(dfno2) 
pm_d_out_m = load_pol(dfpm)
# Avg concentation
no2conc=dataframes['pollutant-concentrations-NO2'].iloc[:,1:10]
pmconc=dataframes['pollutant-concentrations-PM25'].iloc[:,1:10]
no2conc= conc_func(no2conc)
pmconc= conc_func(pmconc)
no2_conc_d_out_dt = pd.merge(no2conc, no2_d_out_m, on=["AreaName", "Time period"])
pm_conc_d_out_dt = pd.merge(pmconc, pm_d_out_m, on=["AreaName", "Time period"])
converttojson(no2_conc_d_out_dt, 'no2pollutant')
converttojson(pm_conc_d_out_dt, 'pm25pollutant')
no2_conc_d_out_dt
pm_conc_d_out_dt

DataFrame 'no2pollutant' saved as JSON file: Clean Data/no2pollutant.json
DataFrame 'pm25pollutant' saved as JSON file: Clean Data/pm25pollutant.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_9172\4236908101.py:264: FutureWarning: The default value of regex will change from True to False in a future version.
  no2_d_out_m["Time period"] = no2_d_out_m["Time period"].str.replace(r' \(%\)$', "")
C:\Users\JKim\AppData\Local\Temp\ipykernel_9172\4236908101.py:264: FutureWarning: The default value of regex will change from True to False in a future version.
  no2_d_out_m["Time period"] = no2_d_out_m["Time period"].str.replace(r' \(%\)$', "")


,AreaName,Time period,Concentration,Exceeding (%)
0,Lambeth,2019,11.2,100.00
1,Inner London,2019,11.3,100.00
2,Outer London,2019,10.3,81.08


In [600]:
# Stop and Search by ethnicity ======================================
stops = dataframes['stop-and-search']
stops['Count']=stops['Count'].astype(int)
# Filter 'Airport' out of Borough of Stop
stops = stops[~stops['Borough of Stop'].str.contains("Airport", na=False)].reset_index(drop=True)
stops['Date'] = pd.to_datetime(stops['Date'])
# Filter Date to 2022
# stops = stops[stops['Date'].dt.year == 2022].reset_index(drop=True)
stops['Time period'] = stops['Date'].dt.year

# Create column Ethnic minority from SDE Group not White
stops['Ethnic minority'] = stops['Self-defined Ethnicity Code'].apply(
    lambda x: 'Not self-defined/unknown' if pd.isna(x) or x.startswith('N') 
    else 'No' if x.startswith('W') 
    else 'Yes'
)

# Aggregate to Borough
stops_b = stops.groupby(['Borough of Stop', 'Ethnic minority', 'Time period'])['Count'].sum().reset_index()

# Rename Borough of Stop to Area name
stops_b.rename(columns={'Borough of Stop': 'AreaName'}, inplace=True)

# Create stops_lam from stops_b
stops_lam = stops_b[stops_b['AreaName'] == "Lambeth"].copy()

# Create stops_lon from sum of stops_b, grouped by Ethnic minority
stops_lon = stops_b.groupby(['Ethnic minority', 'Time period'])['Count'].sum().reset_index()

# Area name = London
stops_lon['AreaName'] = "London"

# Combine stops_lam and stops_lon
stops_b_out = pd.concat([stops_lam, stops_lon], ignore_index=True)
stops_b_out

eth_d = dataframes['ethnicity']
eth_d = eth_d[eth_d['AreaName'].isin(['Lambeth','London'])]
# Select columns that don't have "(%)" in their names
selected_columns = [col for col in eth_d.columns if "(%)" not in col]

# Create a new DataFrame with selected columns
eth_d = eth_d[selected_columns]

eth_d = pd.melt(eth_d, id_vars = 'AreaName', var_name='Ethnic group', value_name='Population')

# Create column 'Ethnic minority' for eth_d, White and not White
eth_d['Ethnic minority'] = eth_d['Ethnic group'].apply(lambda x: 'Yes' if 'White' in x else 'No')

# Aggregate Population by 'Ethnic minority' and 'Area name' in eth_d
eth_d_t = eth_d.groupby(['Ethnic minority', 'AreaName'])['Population'].sum().reset_index()

# Merge eth_d_t with stops_b_out
stops_b_out_m = pd.merge(stops_b_out, eth_d_t, on=['AreaName', 'Ethnic minority'], how='left')

# Calculate stop and search rate
stops_b_out_m['Rate'] = ((stops_b_out_m['Count'] / stops_b_out_m['Population']) * 1000).astype(float).round(1)

# Rename 'Rate' to 'Number of stop and searches per 1,000 people'
stops_b_out_m.rename(columns={'Rate': "Number of 'stop and searches' per 1,000 people"}, inplace=True)
converttojson(stops_b_out_m, 'stopandsearchbyethnicity')
stops_b_out_m

DataFrame 'stopandsearchbyethnicity' saved as JSON file: Clean Data/stopandsearchbyethnicity.json


,AreaName,Ethnic minority,Time period,Count,Population,"Number of 'stop and searches' per 1,000 people"
0,Lambeth,No,2022,1355,142882,9.5
1,Lambeth,No,2023,959,142882,6.7
2,Lambeth,Not self-defined/unknown,2022,2901,NaN,NaN
3,Lambeth,Not self-defined/unknown,2023,2636,NaN,NaN
4,Lambeth,Yes,2022,3695,174781,21.1
5,Lambeth,Yes,2023,3041,174781,17.4
6,London,No,2022,57014,4068548,14.0
7,London,No,2023,44775,4068548,11.0
8,London,Not self-defined/unknown,2022,55764,NaN,NaN
9,London,Not self-defined/unknown,2023,48324,NaN,NaN


In [554]:
# Tree Canopy Cover for 2018 ===============================================
tree=dataframes['tree-canopy-cover']
tree[['lsoa_kmsq','canopy_kmsq','canopy_per']]= tree[['lsoa_kmsq','canopy_kmsq','canopy_per']].apply(pd.to_numeric, errors='coerce')

# Calculate total sum of canopy_per and lsoa_kmsq for tree_lon
canopy_kmsq = tree['canopy_kmsq'].sum()
lsoa_kmsq = tree['lsoa_kmsq'].sum()

# Create DataFrame with the total sums
tree_lon = pd.DataFrame({'canopy_kmsq': [canopy_kmsq], 'lsoa_kmsq': [lsoa_kmsq], 'AreaName': ['London']})

# Create column canopy_per
tree_lon['canopy_per'] = round((tree_lon['canopy_kmsq'] / tree_lon['lsoa_kmsq']) * 100, 2)

# Read inner london boroughs CSV file
inner_london = pd.read_csv("../Data/inner_london_boroughs.csv")

# Strip whitespace from Borough
inner_london['Borough'] = inner_london['Borough'].str.strip()

# Merge inner_london to tree
tree_m = pd.merge(tree, inner_london, left_on='la_nm', right_on='Borough', how='left')

tree_m['Position'] = tree_m['Position'].astype(str)

# Create inner london tree dataframe
tree_in = tree_m[tree_m['Position'] == "Inner London"].groupby(['Position']).agg({'canopy_kmsq': 'sum', 'lsoa_kmsq': 'sum'}).reset_index()

# Create column canopy_per
tree_in['canopy_per'] = round((tree_in['canopy_kmsq'] / tree_in['lsoa_kmsq']) * 100, 2)

# Create column Area name
tree_in['AreaName'] = "Inner London"

# Filter tree to Lambeth
tree_lam = tree[tree['la_nm'] == "Lambeth"].groupby(['la_nm']).agg({'canopy_kmsq': 'sum', 'lsoa_kmsq': 'sum'}).reset_index()

# Create column canopy_per
tree_lam['canopy_per'] = round((tree_lam['canopy_kmsq'] / tree_lam['lsoa_kmsq']) * 100, 2)

# Create column Area name
tree_lam['AreaName'] = "Lambeth"

# Combine Lambeth, Inner London, and London data
tree_f = pd.concat([tree_lam, tree_in, tree_lon], ignore_index=True)

# Make area name an ordered factor
tree_f['AreaName'] = pd.Categorical(tree_f['AreaName'], categories=["Lambeth", "Inner London", "London"], ordered=True)

# Rename columns
tree_f.rename(columns={'canopy_per': 'Percentage of tree canopy cover', 
                        'lsoa_kmsq': 'Total square km',
                        'canopy_kmsq': 'Total square km of tree canopy'}, inplace=True)

# Round columns
tree_f[['Total square km', 'Total square km of tree canopy']] = tree_f[['Total square km', 'Total square km of tree canopy']].round(0)
tree_f=tree_f.iloc[:,1:-1]
converttojson(tree_f,'tree-canopy-cover')
tree_f

DataFrame 'tree-canopy-cover' saved as JSON file: Clean Data/tree-canopy-cover.json


,Total square km of tree canopy,Total square km,Percentage of tree canopy cover,AreaName
0,4327299.0,2.725290e+07,15.88,Lambeth
1,55232828.0,3.069303e+08,18.00,Inner London
2,335866707.0,1.594701e+09,21.06,London


In [519]:
epc_lam = dataframes['energy-performance-certificate-ratings-Lambeth'].iloc[:,1:]
epc_lam = epc_lam[epc_lam['Region']=='Lambeth']
epc_lam = epc_lam[epc_lam['Quarter'] >= '2023/1']
epc_lam.rename(columns={'Region':'AreaName'},inplace = True)
epc_lam['A-C']= epc_lam['A'] + epc_lam['B'] + epc_lam['C']
epc_lam['D-G']= epc_lam['D'] + epc_lam['E'] + epc_lam['F'] + epc_lam['G']
epc_lam['% A-C']= (epc_lam['A-C']/epc_lam['Number Lodgements']*100).astype(float).round(1)
epc_lam

epc_lon = dataframes['energy-performance-certificate-ratings-London']
epc_lon = epc_lon[epc_lon['Region']=='London']
epc_lon = epc_lon[epc_lon['Quarter'] >= '2023/1']
epc_lon.rename(columns={'Region':'AreaName'},inplace = True)
epc_lon['A-C']= epc_lon['A'] + epc_lon['B'] + epc_lon['C']
epc_lon['D-G']= epc_lon['D'] + epc_lon['E'] + epc_lon['F'] + epc_lon['G']
epc_lon['% A-C']= (epc_lon['A-C']/epc_lon['Number Lodgements']*100).astype(float).round(1)
epc_lon

epc_eng = dataframes['energy-performance-certificate-ratings-England'].iloc[:,1:]
epc_eng = epc_eng[epc_eng['Quarter'] >= '2023/1']
epc_eng.rename(columns={'Region':'AreaName'},inplace = True)
epc_eng['A-C']= epc_eng['A'] + epc_eng['B'] + epc_eng['C']
epc_eng['D-G']= epc_eng['D'] + epc_eng['E'] + epc_eng['F'] + epc_eng['G']
epc_eng['% A-C']= (epc_eng['A-C']/epc_eng['Number Lodgements']*100).astype(float).round(1)
epc_eng['AreaName']='England'

epc = pd.concat([epc_lam,epc_lon,epc_eng])
epc.rename(columns={'Quarter':'Time period'}, inplace=True)
epc.drop(columns=['Total Floor Area', 'Not Recorded'], inplace=True)
converttojson(epc, 'EPC_ratings')
epc

,AreaName,Quarter,Number Lodgements,Total Floor Area,A,B,C,D,E,F,G,Not Recorded,A-C,D-G,% A-C
9699,Lambeth,2023/1,2440,176346,5,306,1242,771,95,14,7,0,1553,887,63.6
9700,Lambeth,2023/2,16511,1094202,1,567,9404,5814,636,52,37,0,9972,6539,60.4
9701,Lambeth,2023/3,2503,193744,2,483,1141,720,134,15,8,0,1626,877,65.0
9702,Lambeth,2023/4,2128,155236,9,399,1120,520,69,10,1,0,1528,600,71.8
9703,Lambeth,2024/1,2793,204591,7,382,1261,975,153,9,6,0,1650,1143,59.1


In [505]:
# Public Green Space Access England ==================================
gs_eng = dataframes['access-to-public-green-space-england'].iloc[:,1:]
gs_eng = gs_eng[gs_eng['Country name'] == "England"]
gs_eng_m = pd.melt(gs_eng, id_vars='Country name', var_name='Metric', value_name='Value')
gs_eng_m.rename(columns={"Country name": "AreaName"}, inplace=True)

#LONDON========================
gs_reg= dataframes['access-to-public-green-space-london'].iloc[:,3:]
gs_reg = gs_reg[gs_reg['Region name'] == "London"]
gs_reg_m = pd.melt(gs_reg, id_vars='Region name', var_name='Metric', value_name='Value')
gs_reg_m.rename(columns={"Region name": "AreaName"}, inplace=True)

#Lambeth ===================================
gs_bor= dataframes['access-to-public-green-space-lambeth'].iloc[:,5:]
gs_bor = gs_bor[gs_bor['LAD name'] == "Lambeth"]
gs_bor_m = pd.melt(gs_bor, id_vars='LAD name', var_name='Metric', value_name='Value')
gs_bor_m.rename(columns={"LAD name": "AreaName"}, inplace=True)

greenspace = pd.concat([gs_eng_m,gs_reg_m,gs_bor_m])
converttojson(greenspace, 'access-to-public-green-space2020')
greenspace

DataFrame 'access-to-public-green-space2020' saved as JSON file: Clean Data/access-to-public-green-space2020.json


,AreaName,Metric,Value
0,England,"Average distance to nearest Park, Public Garde...",385.46
1,England,"Average size of nearest Park, Public Garden, o...",94586.38
2,England,"Average number of Parks, Public Gardens, or P...",4.43
3,England,"Average combined size of Parks, Public Garden...",379882.3
0,London,"Average distance to nearest Park, Public Garde...",291.94
1,London,"Average size of nearest Park, Public Garden, o...",158663.29
2,London,"Average number of Parks, Public Gardens, or P...",6.74
3,London,"Average combined size of Parks, Public Garden...",740644.3
0,Lambeth,"Average distance to nearest Park, Public Garde...",292.05
1,Lambeth,"Average size of nearest Park, Public Garden, o...",155337.72


In [508]:
# Solar panel installations ============================================
renew=dataframes['solar-panel-installations']
# Remove spaces at end of column names
renew.columns = renew.columns.str.rstrip()
# Keep only selected columns
renew = renew[['Country', 'Region', 'Local Authority Name', 'Estimated number of households', 'Photovoltaics']]

# London/England --------------------------------------------------------------
renew_lon = solarpanelclean(renew,'Region','London')
renew_eng = solarpanelclean(renew,'Country','England')
renew_lam = solarpanelclean(renew,'Local Authority Name','Lambeth')
renew_f = pd.concat([renew_lam, renew_lon, renew_eng])

# Make area name an ordered factor
renew_f['AreaName'] = pd.Categorical(renew_f['AreaName'], categories=["Lambeth", "London", "England"], ordered=True)

# Remove columns Local Authority Name, Region, Country
renew_f = renew_f[['AreaName', 'Estimated number of households', 'Photovoltaics', 'Number of solar PV installations per 1k households']]

# Rename Photovoltaics to Total number of solar PV installations in area
renew_f.rename(columns={'Photovoltaics': 'Total number of solar PV installations in area'}, inplace=True)
renew_f['Time period'] = 2022

# Last year data ================================================
lyrenew=lydataframes['lastyearsolar-panel-installations']
lyrenew.columns = lyrenew.columns.str.rstrip()
lyrenew = lyrenew[['Country', 'Region', 'Local Authority Name', 'Estimated number of households', 'Photovoltaics']]

# London/England --------------------------------------------------------------
renew_lon = solarpanelclean(lyrenew,'Region','London')
renew_eng = solarpanelclean(lyrenew,'Country','England')
renew_lam = solarpanelclean(lyrenew,'Local Authority Name','Lambeth')
lyrenew_f = pd.concat([renew_lam, renew_lon, renew_eng])

lyrenew_f['AreaName'] = pd.Categorical(lyrenew_f['AreaName'], categories=["Lambeth", "London", "England"], ordered=True)

lyrenew_f = lyrenew_f[['AreaName', 'Estimated number of households', 'Photovoltaics', 'Number of solar PV installations per 1k households']]

lyrenew_f.rename(columns={'Photovoltaics': 'Total number of solar PV installations in area'}, inplace=True)
lyrenew_f['Time period'] = 2021

#Combine years
renew_f = pd.concat([renew_f, lyrenew_f])
renew_f.iloc[:,1]=renew_f.iloc[:,1].astype(int)
converttojson(renew_f, 'solar-panel-installations')
renew_f

DataFrame 'solar-panel-installations' saved as JSON file: Clean Data/solar-panel-installations.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\4280842958.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renew[['Photovoltaics', 'Estimated number of households']] = renew[['Photovoltaics', 'Estimated number of households']].apply(pd.to_numeric)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\4280842958.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renew[['Photovoltaics', 'Estimated number of households']] = renew[['Photovoltaics', 'Estimated number of households']].apply(pd.to_numeric)
C:

,AreaName,Estimated number of households,Total number of solar PV installations in area,Number of solar PV installations per 1k households,Time period
0,Lambeth,128400,1169,9.10,2022
0,London,3502400,51172,14.61,2022
0,England,23077300,1016386,44.04,2022
0,Lambeth,128400,1069,8.33,2021
0,London,3502400,45467,12.98,2021
0,England,23077300,910261,39.44,2021


In [10]:
# Population projections  2020-2030 ==============================
df = dataframes['population-projection-London']#.iloc[9:20, :]
df = df.iloc[:, [1,3]+list(range(15,36))]

df['ward name'].unique()
dflondon = df[df['ward name'].isin(['City of London', 'Borough Total'])]
dflondon['la name'].unique()

dflondon = dflondon.groupby(['la name'], as_index=False).sum()

dflambeth = dflondon[dflondon['la name']=='Lambeth']

dflambeth = pd.melt(dflambeth, id_vars='la name', var_name='Time period', value_name='Value')
dflambeth[['Time period','Value']]=dflambeth[['Time period','Value']].astype(int)
dflambeth.rename(columns= {'la name':'AreaName'}, inplace=True)

# Select only numeric columns
numeric_columns = dflondon.select_dtypes(include='number')
# Sum all numeric columns
column_sums = numeric_columns.sum()

dflondon = pd.DataFrame(column_sums, columns=['Value'])
dflondon.reset_index(inplace=True)
dflondon.rename(columns={dflondon.columns[0]: 'Time period'}, inplace=True)
dflondon[['Time period','Value']]=dflondon[['Time period','Value']].astype(int)
# Add a column 'AreaName' with value 'London'
dflondon['AreaName'] = 'London'

warddf= df[df['ward name'].isin(wards)]
warddf = warddf.groupby(['ward name'], as_index=False).sum()
warddf = pd.melt(warddf, id_vars='ward name', var_name='Time period', value_name='Value')
warddf[['Time period','Value']]=warddf[['Time period','Value']].astype(int)
warddf.rename(columns= {'ward name':'AreaName'}, inplace=True)

popprojection = pd.concat([dflambeth,dflondon,warddf])
converttojson(popprojection, 'populationprojection2020-40')
popprojection

DataFrame 'populationprojection2020-40' saved as JSON file: Clean Data/populationprojection2020-40.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_16428\823314276.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dflondon = dflondon.groupby(['la name'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_16428\823314276.py:30: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  warddf = warddf.groupby(['ward name'], as_index=False).sum()


,AreaName,Time period,Value
0,Lambeth,2020,322949
1,Lambeth,2021,317334
2,Lambeth,2022,322078
3,Lambeth,2023,324637
4,Lambeth,2024,325838
...,...,...,...
520,Streatham St Leonard's,2040,17440
521,Streatham Wells,2040,10855
522,Vauxhall,2040,12496
523,Waterloo & South Bank,2040,13725


In [455]:
rec=dataframes['recycling-rates']
# Replace . in column names
rec.columns = rec.columns.str.replace(".", " ")

# Filter to financial years 2020/21-2021/22
rec = rec[rec['Year'].isin(["2020-21",'2021-22'])]

# Filter to local authority containing Lambeth
rec_lam = rec[rec['Authority'].str.contains("Lambeth")]

# Keep selected columns
rec_lam = rec_lam[['Year', 'Authority', 'Percentage of household waste sent for reuse, recycling or composting (Ex NI192)']]

# Rename Authority to Area name
rec_lam.rename(columns={'Authority': 'AreaName', 'Year': 'Time period'}, inplace=True)

# Change Lambeth LB to Lambeth
rec_lam['AreaName'] = rec_lam['AreaName'].str.replace(" LB", "")

# Convert percentage to decimal
rec_lam['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'] *= 100
rec_lam['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'] = rec_lam['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'].astype(float).round(1)
rec_lam

lonrec=dataframes['recycling-rates-london']
# Replace . in column names
lonrec.columns = lonrec.columns.str.replace(".", " ")

# Filter to financial year 2020/21-2021/22
lonrec = lonrec[['Region', '2020-21','2021-22']]

# Filter to local authority containing Lambeth
rec_lon = lonrec[lonrec['Region'].str.contains("London|England")]

# Rename Authority to Area name
rec_lon.rename(columns={'Region': 'AreaName'}, inplace=True)

rec_lon = pd.melt(rec_lon, id_vars='AreaName', var_name='Time period', value_name='Percentage of household waste sent for reuse, recycling or composting (Ex NI192)')

# Convert percentage to decimal
# rec_lon['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'] *= 100
# rec_lon['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'] = rec_lon['Percentage of household waste sent for reuse, recycling or composting (Ex NI192)'].astype(float).round(1)

recycling = pd.concat([rec_lam, rec_lon])
converttojson(recycling, 'recycling-rates')
recycling

DataFrame 'recycling-rates' saved as JSON file: Clean Data/recycling-rates.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3808643449.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rec.columns = rec.columns.str.replace(".", " ")
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3808643449.py:27: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lonrec.columns = lonrec.columns.str.replace(".", " ")
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3808643449.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_lon.rename(columns={'Region': 'AreaName'}, inplace=Tr

,Time period,AreaName,"Percentage of household waste sent for reuse, recycling or composting (Ex NI192)"
3701,2020-21,Lambeth,36.4
4046,2021-22,Lambeth,34.5
0,2020-21,London,33
1,2020-21,England,42.3
2,2021-22,London,32.7
3,2021-22,England,42.5


In [427]:
# Greenhouse Gas emissions
ghg = dataframes['greenhouse-gas-emissions']
# Replace . in column names
ghg.columns = ghg.columns.str.replace(".", " ")

# Filter to calendar year on or later than 2018 (latest data available)
ghg = ghg[ghg['Calendar Year'] >= 2020]

# Filter Local Authority to Lambeth, London Total, and England Total
ghg_f = ghg[ghg['Local Authority'].isin(["Lambeth", "London Total", "England Total"])]

# Keep only columns of interest
ghg_f = ghg_f[['Local Authority', 'Calendar Year', 'Grand Total', 'Per Capita Emissions (tCO2e)']]

# Convert Per Capita Emissions and Grand Total to numeric and Round to 2 decimal places
ghg_f['Per Capita Emissions (tCO2e)'] = ghg_f['Per Capita Emissions (tCO2e)'].astype(float).round(2)
ghg_f['Grand Total'] = ghg_f['Grand Total'].astype(float).round()

# Replace Total in Local Authority
ghg_f['Local Authority'] = ghg_f['Local Authority'].str.replace(" Total", "")

# Rename Local Authority to Area name
ghg_f.rename(columns={'Local Authority': 'AreaName'}, inplace=True)

# Rename Calendar Year to Year
ghg_f.rename(columns={'Calendar Year': 'Time period'}, inplace=True)

# Make Area name an ordered factor with levels Lambeth, London, England
ghg_f['AreaName'] = pd.Categorical(ghg_f['AreaName'], categories=["Lambeth", "London", "England"], ordered=True)

# Rename Grand Total to Total GHG emissions (ktCO2e)
ghg_f.rename(columns={'Grand Total': 'Total GHG emissions (ktCO2e)'}, inplace=True)
converttojson(ghg_f, 'greenhousegas_emissions')
ghg_f

DataFrame 'greenhousegas_emissions' saved as JSON file: Clean Data/greenhousegas_emissions.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\387061211.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ghg.columns = ghg.columns.str.replace(".", " ")


,AreaName,Time period,Total GHG emissions (ktCO2e),Per Capita Emissions (tCO2e)
3874,Lambeth,2020,870.0,2.68
3875,Lambeth,2021,926.0,2.92
4078,London,2020,27476.0,3.11
4079,London,2021,29948.0,3.40
5897,England,2020,291927.0,5.16
5898,England,2021,309040.0,5.47


In [360]:
reoff=dataframes['reoffending-rates']
reoff=reoff[reoff['Adult / Juvenile']=='Adult']
reoff.columns = reoff.columns.str.replace(".", " ")
# Filter rows
reoff_f = reoff[reoff['Category'].str.contains("total") & (reoff['Adult / Juvenile'] == "Adult") &
                (reoff['Geography'].str.startswith("England") | reoff['Geography'].str.startswith("London") |
                 reoff['Geography'].str.contains("Lambeth")) & (reoff['Category'] != "UTLA total")]

# Extract year from Cohort column
reoff_f['Year'] = reoff_f['Cohort'].str.extract(r'(\d\d\d\d$)')

# Select columns
reoff_f = reoff_f[['Geography', 'Year', 'offenders', 'reoffenders']]

# Calculate Percentage of offenders who reoffend
reoff_f['Percentage of offenders who reoffend'] = ((reoff_f['reoffenders'] / reoff_f['offenders']) * 100).astype(float).round(1)

# Filter rows for year >= 2017
reoff_f_17 = reoff_f[reoff_f['Year'] >= '2021']
# Rename columns
reoff_f_17.columns = ["Area name", "Year", "Number of offenders", "Number of reoffenders",
                      "Percentage of offenders who reoffended (%)"]

# Make 'Area name' an ordered factor with Lambeth, London, England
reoff_f_17['Area name'] = pd.Categorical(reoff_f_17['Area name'], categories=["Lambeth", "London", "England and Wales"], ordered=True)
converttojson(reoff_f_17, 'reoffending-rates')
reoff_f_17

C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\592352327.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reoff.columns = reoff.columns.str.replace(".", " ")


DataFrame 'reoffending-rates' saved as JSON file: Clean Data/reoffending-rates.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\592352327.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reoff_f['Year'] = reoff_f['Cohort'].str.extract(r'(\d\d\d\d$)')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\592352327.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reoff_f_17['Area name'] = pd.Categorical(reoff_f_17['Area name'], categories=["Lambeth", "London", "England and Wales"], ordered=True)


,Area name,Year,Number of offenders,Number of reoffenders,Percentage of offenders who reoffended (%)
235590,England and Wales,2021,288877,69281,24.0
243598,Lambeth,2021,2006,517,25.8
246245,London,2021,43156,10242,23.7
258407,England and Wales,2022,317283,79004,24.9
266420,Lambeth,2022,2217,511,23.0
269069,London,2022,45381,10285,22.7


In [329]:
# Public perception of Police 
dflondon=dataframes['public-perception-of-police'].iloc[26:,[0,-1]]
dflondon = pd.melt(dflondon, id_vars=['Date'], var_name='Opinion', value_name='Percent')
dflondon.rename(columns = {'Date':'Time period'}, inplace=True)
dflondon['Percent'] *= 100
dflondon['AreaName'] = 'London (MPS)'
dflondon

df=dataframes['public-perception-of-police-lambeth'].iloc[26:,[0,-1]]
df = pd.melt(df, id_vars=['Date'], var_name='Opinion', value_name='Percent')
df.rename(columns = {'Date':'Time period'}, inplace=True)
df['Percent'] *= 100
df['AreaName'] = 'Lambeth'
df

df = pd.concat([dflondon,df])
converttojson(df, 'public-perception-police')
df

DataFrame 'public-perception-police' saved as JSON file: Clean Data/public-perception-police.json


,Time period,Opinion,Percent,AreaName
0,June 2021,Police do a good job in the local area,54.0,London (MPS)
1,September 2021,Police do a good job in the local area,52.0,London (MPS)
2,December 2021,Police do a good job in the local area,51.0,London (MPS)
3,March 2022,Police do a good job in the local area,49.0,London (MPS)
4,June 2022,Police do a good job in the local area,49.0,London (MPS)
0,June 2021,Police do a good job in the local area,49.0,Lambeth
1,September 2021,Police do a good job in the local area,48.0,Lambeth
2,December 2021,Police do a good job in the local area,45.0,Lambeth
3,March 2022,Police do a good job in the local area,40.0,Lambeth
4,June 2022,Police do a good job in the local area,43.0,Lambeth


In [297]:
traffic = dataframes['traffic-incidents']
lytraffic =lydataframes['lastyeartraffic-incidents']
traffic_b_out_m = trafficclean(traffic, 'Jan-Sept 2023')
lytraffic_b_out_m = trafficclean(lytraffic, 'Jan-Sept 2022')

traffic = pd.concat([lytraffic_b_out_m, traffic_b_out_m])
converttojson(traffic, 'traffic-incidents')
traffic

DataFrame 'traffic-incidents' saved as JSON file: Clean Data/traffic-incidents.json


,AreaName,Number of traffic incidents casualties,Population,"Number of traffic incidents casualties per 1,000 people",Time period
0,Lambeth,1297,317654,4.1,Jan-Sept 2022
1,London,27207,8799728,3.1,Jan-Sept 2022
0,Lambeth,875,317654,2.8,Jan-Sept 2023
1,London,19606,8799728,2.2,Jan-Sept 2023


In [222]:
#Electoral Registrations 2022 and 2023 =================================================
df=dataframes['electoral-registration'].iloc[:,1:4]
df.rename(columns={'Local Government Areas':'AreaName', df.columns[1]:'2022', df.columns[2]:'2023'}, inplace=True)
# df['AreaName']=df['AreaName'].replace(' and ', ' & ')
df= df[df['AreaName'].isin(['Lambeth','London','ENGLAND'])]
df['AreaName']=df['AreaName'].replace('ENGLAND', 'England')
df = pd.melt(df, id_vars=['AreaName'], var_name='Time period', value_name='Value')
converttojson(df, 'electoral-registration')
df

DataFrame 'electoral-registration' saved as JSON file: Clean Data/electoral-registration.json


,AreaName,Time period,Value
0,England,2022,40845079
1,London,2022,5998415
2,Lambeth,2022,222324
3,England,2023,41032338
4,London,2023,6016888
5,Lambeth,2023,221948


In [223]:
# Number and % of people over 65 on pension credit
df= dataframes['pension-credit'].iloc[1:-9,1:]
df.rename(columns= {df.columns[0]:'AreaName'}, inplace=True)
df = pd.melt(df, id_vars='AreaName', var_name='Time period', value_name='Value')
df['AreaName'] = df['AreaName'].str.replace(' and ', ' & ')
merged_df = pd.merge(df, over65, on='AreaName')
merged_df['Value'] = pd.to_numeric(merged_df['Value'], errors='coerce')
merged_df['Percent'] = (merged_df['Value']/merged_df['Aged over 65 (2020)'] *100).round(1)
converttojson(merged_df, 'people-over-65-on-pension-credit')
merged_df

DataFrame 'people-over-65-on-pension-credit' saved as JSON file: Clean Data/people-over-65-on-pension-credit.json


,AreaName,Time period,Value,Aged over 65 (2020),Percent
0,Lambeth,Aug-19,7883,27265,28.9
1,Lambeth,Aug-20,7534,27265,27.6
2,Lambeth,Aug-21,7329,27265,26.9
3,Lambeth,Aug-22,7254,27265,26.6
4,Lambeth,Aug-23,7501,27265,27.5
...,...,...,...,...,...
135,England,Aug-19,1329979,10401300,12.8
136,England,Aug-20,1261761,10401300,12.1
137,England,Aug-21,1202132,10401300,11.6
138,England,Aug-22,1165926,10401300,11.2


In [224]:
# Children in poverty BHC
df=dataframes['children-in-low-income-households-BHC-Lambeth'].iloc[:, [0,18,19]]
df.rename(columns={df.columns[0]:'AreaName',
                        df.columns[1]:'2022',
                        df.columns[2]:'2023'}, inplace=True)

df = df[df.iloc[:,0] == 'Lambeth']

londondf=dataframes['children-in-low-income-households-BHC-London'].iloc[:, [0,18,19]]
londondf.rename(columns={londondf.columns[0]:'AreaName',
                        londondf.columns[1]:'2022',
                        londondf.columns[2]:'2023'}, inplace=True)
londondf['AreaName'] = londondf['AreaName'].str.replace(r' \[note 4\]', '')
londondf = londondf[londondf.iloc[:,0].isin(['London', 'United Kingdom'])]

mergeddf = pd.concat([londondf,df], ignore_index=True)
mergeddf = pd.melt(mergeddf, id_vars='AreaName', var_name='Time period', value_name='Percent')
mergeddf['Percent'] *= 100 
converttojson(mergeddf, 'children-in-low-income-households-BHC')
mergeddf
# Ward data not available yet -----------------------------------
# wardsdf=dataframes['children-in-low-income-households-BHC-wards'].iloc[:, [0,2,20,21]]
# wardsdf = wardsdf.rename(columns={wardsdf.columns[0]:'AreaName'})
# wardsdf.iloc[:,1] = wardsdf.iloc[:,1].str.replace(' and ', ' & ')
# wardsdf = wardsdf[wardsdf.iloc[:,0]== 'Lambeth']
# wardsdf= wardsdf.iloc[:, 1:]
# wardsdf

DataFrame 'children-in-low-income-households-BHC' saved as JSON file: Clean Data/children-in-low-income-households-BHC.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2761574122.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  londondf['AreaName'] = londondf['AreaName'].str.replace(r' \[note 4\]', '')


,AreaName,Time period,Percent
0,United Kingdom,2022,20.1
1,London,2022,16.4
2,Lambeth,2022,18.2
3,United Kingdom,2023,20.1
4,London,2023,15.8
5,Lambeth,2023,18.0


In [225]:
#Poverty rate AHC ------------------------------------------
# 5year avg data from 2016/17-2021/22 excluding 2020/21
lambethdf = dataframes['poverty-rate-lambeth']
lambethdf=lambethdf[lambethdf['London borough']=='Lambeth'].loc[:,['London borough', 'Poverty rate (AHC)']]
lambethdf.rename(columns={'London borough':'AreaName', 'Poverty rate (AHC)': 'Percent'}, inplace=True)
lambethdf['Time period']= '2016/17-2021-22'
lambethdf['Percent'] = int(lambethdf['Percent'].str.replace("%", ""))

df = dataframes['poverty-rate-londonengland']

df = df[df['Region'].isin(['London', 'England']) &
       (df['Group']=='Total')&
       (df['Poverty measure']=='Relative')]

df = df[(df['Quantity']=='%') &
        (df['Housing costs']=='AHC')]
df = df.loc[:,['Region', '2019/20 - 2021/22', '2020/21 - 2022/23']]
df
melted_df = pd.melt(df, id_vars=['Region'], var_name='Time period', value_name='Percent')
# 
melted_df.rename(columns={'Region':'AreaName'}, inplace=True)

combineddf = pd.concat([lambethdf, melted_df], ignore_index=True)
converttojson(combineddf, 'poverty-rate-AHC')
combineddf

DataFrame 'poverty-rate-AHC' saved as JSON file: Clean Data/poverty-rate-AHC.json


,AreaName,Percent,Time period
0,Lambeth,28,2016/17-2021-22
1,England,22,2019/20 - 2021/22
2,London,25,2019/20 - 2021/22
3,England,22,2020/21 - 2022/23
4,London,24,2020/21 - 2022/23


In [226]:
# Perception of Safety Resident survey ----------------------------------------------------------
q015_1 = dataframes['resident-survey-perception-of-safety-day']
q015_2= dataframes['resident-survey-perception-of-safety-evening']
q1y22 = dataframes['resident-survey-police-confidence']
q2bnew22 = dataframes['resident-survey-harassment-hate-crime']

# Join all DataFrames
surv_sat_f_safety = pd.concat([q015_1, q015_2, q1y22, q2bnew22], ignore_index=True)

# Add a 'Year' column with value 2023
surv_sat_f_safety['Time period'] = 2023
surv_sat_f_safety.drop(surv_sat_f_safety.columns[2], axis=1, inplace=True)

# Remove anything between square brackets in Responses
surv_sat_f_safety['Response'] = surv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_safety['Response'] = pd.Categorical(surv_sat_f_safety['Response'], categories=surv_sat_f_safety['Response'].unique(), ordered=True)
converttojson(surv_sat_f_safety,'survey_wellbeing_safety')
surv_sat_f_safety

# Last Year Perception of Safety Resident survey ----------------------------------------------------------
q015_1 = lydataframes['lastyearresident-survey-perception-of-safety-day']
q015_2= lydataframes['lastyearresident-survey-perception-of-safety-evening']
q1y22 = lydataframes['lastyearresident-survey-police-confidence']

# Join all DataFrames
lastyearsurv_sat_f_safety = pd.concat([q015_1, q015_2, q1y22], ignore_index=True)

# Add a 'Year' column with value 2022
lastyearsurv_sat_f_safety['Time period'] = 2022

# Remove anything between square brackets in Responses
lastyearsurv_sat_f_safety['Response'] = lastyearsurv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lastyearsurv_sat_f_safety['Response'] = pd.Categorical(lastyearsurv_sat_f_safety['Response'], categories=lastyearsurv_sat_f_safety['Response'].unique(), ordered=True)
converttojson(lastyearsurv_sat_f_safety,'lastyearsurvey_wellbeing_safety')
lastyearsurv_sat_f_safety

DataFrame 'survey_wellbeing_safety' saved as JSON file: Clean Data/survey_wellbeing_safety.json
DataFrame 'lastyearsurvey_wellbeing_safety' saved as JSON file: Clean Data/lastyearsurvey_wellbeing_safety.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\1209231986.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_safety['Response'] = surv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\1209231986.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  lastyearsurv_sat_f_safety['Response'] = lastyearsurv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')


,Response,Percent,Question,Time period
0,Very safe,47.9,"To what extent would you say you are, or would...",2022
1,Fairly safe,44.2,"To what extent would you say you are, or would...",2022
2,Not particularly safe,5.9,"To what extent would you say you are, or would...",2022
3,Not safe at all,1.3,"To what extent would you say you are, or would...",2022
4,Very safe,14.7,"To what extent would you say you are, or would...",2022
5,Fairly safe,48.5,"To what extent would you say you are, or would...",2022
6,Not particularly safe,24.3,"To what extent would you say you are, or would...",2022
7,Not safe at all,10.5,"To what extent would you say you are, or would...",2022
8,Strongly agree,8.5,To what extent do you agree or disagree that y...,2022
9,Tend to agree,25.6,To what extent do you agree or disagree that y...,2022


In [245]:
df=dataframes['overall-crime-rate']
df = crimerate(df)
lydf=lydataframes['lastyearoverall-crime-rate']
lydf = crimerate(lydf)
overallcrime = pd.concat([df, lydf], ignore_index=True)
converttojson(overallcrime,'monthlyoverallcrimespercapita')
overallcrime

df=dataframes['hate-crime-rate']
df = crimerate(df, 'Hate crime')
lydf=lydataframes['lastyearhate-crime-rate']
lydf = crimerate(lydf, 'Hate crime')
hate = pd.concat([df, lydf], ignore_index=True)
converttojson(hate,'monthlyhatecrimespercapita')
hate

df=dataframes['domestic-abuse-crime-rate']
df = crimerate(df, 'Domestic Abuse')
lydf=lydataframes['lastyeardomestic-abuse-crime-rate']
lydf = crimerate(lydf, 'Domestic Abuse')
abuse = pd.concat([df, lydf], ignore_index=True)
converttojson(abuse,'monthlydomesticabusecrimespercapita')
abuse

df=dataframes['knife-crime-rate']
df = crimerate(df, 'Knife crime')
lydf=lydataframes['lastyearknife-crime-rate']
lydf = crimerate(lydf, 'Knife crime')
knives = pd.concat([df, lydf], ignore_index=True)
converttojson(knives,'monthlyknifecrimespercapita')
knives

df=dataframes['knife-crime-rate']
df = crimerate(df, 'Knife crime', "Knife Injury Victims (non DA 1-24)")
lydf=lydataframes['lastyearknife-crime-rate']
lydf = crimerate(lydf, 'Knife crime', "Knife Injury Victims (non DA 1-24)")
knives = pd.concat([df, lydf], ignore_index=True)
converttojson(knives,'monthlyknifecrimesvictimsU25percapita')
knives['AreaName'].unique()
knives

C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlyoverallcrimespercapita' saved as JSON file: Clean Data/monthlyoverallcrimespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlyhatecrimespercapita' saved as JSON file: Clean Data/monthlyhatecrimespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


...

DataFrame 'monthlyknifecrimespercapita' saved as JSON file: Clean Data/monthlyknifecrimespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlyknifecrimesvictimsU25percapita' saved as JSON file: Clean Data/monthlyknifecrimesvictimsU25percapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


array(['Lambeth', 'Brixton Acre Lane', 'Brixton North',
       'Brixton Rush Common', 'Brixton Windrush', 'Clapham East',
       'Clapham Town', 'Gipsy Hill', 'Herne Hill & Loughborough Junction',
       "Knight's Hill", "Myatt's Fields", 'Oval', "St Martin's",
       'Stockwell East', 'Stockwell West & Larkhall',
       'Streatham Hill East', 'Streatham Wells', 'Vauxhall',
       'Waterloo & South Bank', 'West Dulwich', 'London',
       'Clapham Common & Abbeville', 'Clapham Park', 'Kennington',
       'Streatham Common & Vale', 'Streatham Hill West & Thornton',
       "Streatham St Leonard's"], dtype=object)

In [228]:
violentcrime = crimerate(dataframes['violent-crime-rate'], 'Violence Against the Person')
lyviolentcrime = crimerate(lydataframes['lastyearviolent-crime-rate'], 'Violence Against the Person')
violentcrime = pd.concat([violentcrime, lyviolentcrime], ignore_index=True)
converttojson(violentcrime,'monthlyviolentcrimespercapita')
lyviolentcrime

burglar2023 = crimerate(dataframes['domestic-burglary-crime-rate'], 'Burglary', 'Domestic Burglary')
burglar2022 = crimerate(lydataframes['lastyeardomestic-burglary-crime-rate'], 'Burglary', 'Domestic Burglary')
burglar = pd.concat([burglar2023, burglar2022], ignore_index=True)
converttojson(burglar,'monthlydomesticburglarypercapita')
burglar

drugs2023 = crimerate(dataframes['drug-and-alcohol-misuse'], 'Drug Offences')
drugs2022 = crimerate(lydataframes['lastyeardrug-and-alcohol-misuse'], 'Drug Offences')
drugs = pd.concat([drugs2023, drugs2022], ignore_index=True)
converttojson(drugs,'monthlydrugoffensespercapita')
drugs

C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlyviolentcrimespercapita' saved as JSON file: Clean Data/monthlyviolentcrimespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlydomesticburglarypercapita' saved as JSON file: Clean Data/monthlydomesticburglarypercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlydrugoffensespercapita' saved as JSON file: Clean Data/monthlydrugoffensespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2735516465.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


,AreaName,Population,Time period,Count,"Offences per 1,000 people"
0,Lambeth,317654,2023-04,104,0.3
1,Lambeth,317654,2023-05,116,0.4
2,Lambeth,317654,2023-06,104,0.3
3,Lambeth,317654,2023-07,113,0.4
4,Lambeth,317654,2023-08,125,0.4
...,...,...,...,...,...
564,London,8799728,2022-11,3531,0.4
565,London,8799728,2022-12,2899,0.3
566,London,8799728,2023-01,3738,0.4
567,London,8799728,2023-02,3047,0.3


In [230]:
# 2018 election
df = lydataframes['lastyeartotal-election-turnout-londonlambeth']
df = df.iloc[0:33,[1,2,6,7]]
df = firstrowtocolname(df)
column_mapping = {df.columns[0]: 'AreaName',
                  df.columns[2]: 'Ballot papers at the count',
                 df.columns[3] : 'Ballot box turnout (%)'}
df.rename(columns=column_mapping, inplace=True)
lydf = df[df['AreaName']=='Lambeth']

dflondon = lydataframes['lastyeartotal-election-turnout-lambethwards']
dflondon = dflondon.iloc[1:,[1,3,5,8,9]]
dflondon = firstrowtocolname(dflondon)

dflondon = dflondon[dflondon['Ward'].isin(['Total London Boroughs', 'England'])]

dflondon=dflondon.drop(dflondon.columns[[1]], axis=1, inplace=False)
dflondon

column_mapping = {dflondon.columns[0]: 'AreaName', 
                  dflondon.columns[2]: 'Ballot papers at the count',
                 dflondon.columns[3] : 'Ballot box turnout (%)'}
dflondon.rename(columns=column_mapping, inplace=True)

election_df2018 = pd.concat([lydf, dflondon], ignore_index=True)
election_df2018['Time period'] = 2018

# 2022 election ---------------------------------------------------------------
df = dataframes['total-election-turnout-londonlambeth']
df = df.iloc[:2,:4]
column_mapping = {df.columns[0]: 'AreaName'}
df.rename(columns=column_mapping, inplace=True)

dfengland = dataframes['total-election-turnout-england']
column_mapping = {dfengland.columns[0]: 'AreaName'}
dfengland.rename(columns=column_mapping, inplace=True)
dfengland = dfengland.iloc[0,:4]

dfwards = dataframes['total-election-turnout-lambethwards']
dfwards = dfwards.iloc[:,1:4]
dfwards['Ballot box turnout'] = dfwards.iloc[:,2].astype(int)/ dfwards.iloc[:,1].astype(int)
column_mapping = {dfwards.columns[0]: 'AreaName'}
dfwards.rename(columns=column_mapping, inplace=True)

election_df = pd.concat([df, dfwards], ignore_index=True)
election_df = election_df.append(dfengland, ignore_index=True)
election_df['Time period'] = 2022
column_mapping = {election_df.columns[2] :'Ballot papers at the count',
                  election_df.columns[3]: 'Ballot box turnout (%)'} # percentage
election_df.rename(columns=column_mapping, inplace=True)

# Combine both years ---------------------------------------------
election_df = pd.concat([election_df, election_df2018], ignore_index=True)
replace_dict = {'London Borough of Lambeth': 'Lambeth', 'Total London': 'London', 'Total':'England',
               'Total London Boroughs':'London'}
election_df['AreaName'] = election_df['AreaName'].replace(replace_dict)

election_df['Ballot box turnout (%)'] = election_df['Ballot box turnout (%)'].apply(lambda x: round(x * 100, 1))

converttojson(election_df, 'election2018-22')
election_df
# election_df2018
# dflondon

DataFrame 'election2018-22' saved as JSON file: Clean Data/election2018-22.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2430298777.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwards['Ballot box turnout'] = dfwards.iloc[:,2].astype(int)/ dfwards.iloc[:,1].astype(int)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2430298777.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  election_df = election_df.append(dfengland, ignore_index=True)


,AreaName,Electorate,Ballot papers at the count,Ballot box turnout (%),Time period
0,Lambeth,226899,71925,31.7,2022
1,London,6036183,2151194,35.6,2022
2,Brixton Acre Lane,11042,3271,29.6,2022
3,Brixton North,11084,2869,25.9,2022
4,Brixton Rush Common,10646,3306,31.1,2022
5,Brixton Windrush,6845,1858,27.1,2022
6,Clapham Common & Abbeville,7357,2908,39.5,2022
7,Clapham East,7439,1744,23.4,2022
8,Clapham Park,9710,2497,25.7,2022
9,Clapham Town,11514,3562,30.9,2022


In [231]:
# Weekly earnings by sex ------------------------------------------------------------------
lydf= lydataframes['lastyearweekly-earnings-by-sex']
lydf.iloc[:,0] = lydf.iloc[:,0].str.split(':').str[-1]
lydf = lydf.iloc[1:,[0,1,3,5]]
lydf['Time period'] = 2022

df = dataframes['weekly-earnings-by-sex']
df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
df = df.iloc[1:,[0,1,3,5]]
df['Time period'] = 2023

filtered_df = pd.concat([lydf, df], ignore_index=True)
column_mapping = {filtered_df.columns[0]: 'AreaName'}
filtered_df.rename(columns=column_mapping, inplace=True)

melted_df = pd.melt(filtered_df, id_vars=['AreaName', 'Time period'], var_name='Sex', value_name='Value')
converttojson(melted_df, 'weekly-earnings-by-sex')
melted_df

DataFrame 'weekly-earnings-by-sex' saved as JSON file: Clean Data/weekly-earnings-by-sex.json


,AreaName,Time period,Sex,Value
0,Lambeth,2022,Male,767.2
1,England,2022,Male,631.6
2,London,2022,Male,718.7
3,Lambeth,2023,Male,NaN
4,England,2023,Male,NaN
5,London,2023,Male,NaN
6,Lambeth,2022,Female,658.4
7,England,2022,Female,450.1
8,London,2022,Female,574.9
9,Lambeth,2023,Female,NaN


In [232]:
# % ppl used Internet in the past 3 months
df=dataframes['internet-users']
df= df.drop(df.columns[[0,2]], axis=1, inplace=False)
df = df.iloc[:,:8]
column_mapping = {df.columns[0]: 'AreaName'}
df.rename(columns=column_mapping, inplace=True)
filtered_df = df[df.iloc[:,0].isin(['Lambeth','UK','London'])]
filtered_df= filtered_df[['AreaName',2019,2020]]
melted_df = pd.melt(filtered_df, id_vars='AreaName', var_name='Time period', value_name='Value')
converttojson(melted_df, 'internetusers2019-20')
melted_df

DataFrame 'internetusers2019-20' saved as JSON file: Clean Data/internetusers2019-20.json


,AreaName,Time period,Value
0,UK,2019,90.8
1,London,2019,93
2,Lambeth,2019,95.2
3,UK,2020,92.1
4,London,2020,94.9
5,Lambeth,2020,94.9


In [13]:
dataframes['sexual-orientation']

,AreaName,Straight or Heterosexual (number),Straight or Heterosexual (%),Gay or Lesbian (number),Gay or Lesbian (%),Bisexual (number),Bisexual (%),All other sexual orientations (number),All other sexual orientations (%),Not answered (number),Not answered (%)
1,Lambeth,222348,82.7,14291,5.3,6063,2.3,1844,0.7,24398,9.1
2,England,41114478,89.4,709704,1.5,591690,1.3,158357,0.3,3432728,7.5
3,London,6123195,86.2,158760,2.2,108140,1.5,37482,0.5,676408,9.5


In [14]:
df = dataframes['sexual-orientation']
df_filtered = df[df.columns[~df.columns.str.contains('Straight')]]
converttojson(df_filtered, 'sexual-orientation')

df = dataframes['gender-identity']
df_filtered = df[df.columns[~df.columns.str.contains('same as sex')]]
converttojson(df_filtered, 'gender-identity')

DataFrame 'sexual-orientation' saved as JSON file: Clean Data/sexual-orientation.json
DataFrame 'gender-identity' saved as JSON file: Clean Data/gender-identity.json


In [48]:
# Working Age Population 16-65 
df= dataframes['working-age-population22'].iloc[:3,:]
df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
df['Time period'] = 2022

lydf = lydataframes['lastyearworking-age-population21']
lydf.iloc[:,0] = lydf.iloc[:,0].str.split(':').str[-1]
lydf['Time period'] = 2021
df = pd.concat([df, lydf], ignore_index=True)
df.rename(columns = {'Area':'AreaName'}, inplace= True) 
df
converttojson(df, 'working-age-population')
df

DataFrame 'working-age-population' saved as JSON file: Clean Data/working-age-population.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_10020\2394251399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
C:\Users\JKim\AppData\Local\Temp\ipykernel_10020\2394251399.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time period'] = 2022


,AreaName,Male,Female,Time period
0,Lambeth,116800,124400,2022
1,England,17696300,18205000,2022
2,London,2960200,3142700,2022
3,Lambeth,117100,124700,2021
4,England,17529300,18088800,2021
5,London,2937200,3125000,2021


In [236]:
df = dataframes['personal-wellbeing']
df= df[['Geography','MeasureOfWellbeing', '2022-23', '2021-22']]
df.rename(columns={df.columns[0]: 'AreaName'}, inplace=True)
melted_df = pd.melt(df, id_vars=['AreaName', 'MeasureOfWellbeing'], var_name='Time period', value_name='Average Value')
converttojson(melted_df, 'personal-wellbeing2021-23')
melted_df

DataFrame 'personal-wellbeing2021-23' saved as JSON file: Clean Data/personal-wellbeing2021-23.json


,AreaName,MeasureOfWellbeing,Time period,Average Value
0,England,Anxiety,2022-23,3.24
1,England,Happiness,2022-23,7.38
2,England,Life satisfaction,2022-23,7.44
3,England,Worthwhile,2022-23,7.73
4,Lambeth,Anxiety,2022-23,3.45
5,Lambeth,Happiness,2022-23,7.5
6,Lambeth,Life satisfaction,2022-23,7.63
7,Lambeth,Worthwhile,2022-23,7.75
8,London,Anxiety,2022-23,3.34
9,London,Happiness,2022-23,7.32


In [237]:
lambeth = dataframes['children-in-low-income-households-AHC-Lambeth'].iloc[:, [1] + list(range(11, 19))]
london = dataframes['children-in-low-income-households-AHC-London'].iloc[:, [0] + list(range(9, 17))]
df = pd.concat([lambeth, london], ignore_index=True)
df.rename(columns={df.columns[0]: 'AreaName'}, inplace=True)
filtered_df = df[df.iloc[:,0].isin(['Lambeth','England','London'])]
filtered_df= filtered_df[['AreaName','2020/21','2021/22']]

melted_df = pd.melt(filtered_df, id_vars=['AreaName'], var_name='Time period', value_name='Percent')
melted_df['Percent'] = melted_df['Percent'].apply(lambda x: round(x * 100, 1) if pd.notnull(x) else x)

converttojson(melted_df, 'children-in-low-income-households-AHC')
melted_df

DataFrame 'children-in-low-income-households-AHC' saved as JSON file: Clean Data/children-in-low-income-households-AHC.json


,AreaName,Time period,Percent
0,Lambeth,2020/21,38.8
1,England,2020/21,30.6
2,London,2020/21,37.2
3,Lambeth,2021/22,35.5
4,England,2021/22,30.8
5,London,2021/22,32.9


In [238]:
# Cost of Living Resident survey ----------------------------------------------------------
q032 = dataframes['resident-survey-results---financial']
q4ay22_1 = dataframes['resident-survey-results---payforfood']
q4ay22_2 = dataframes['resident-survey-results---payforenergy']
q4ay22_3 = dataframes['resident-survey-results---payforrent']

# Join all DataFrames
surv_sat_f_cost = pd.concat([q032, q4ay22_1, q4ay22_2, q4ay22_3], ignore_index=True)

# Add a 'Year' column with value 2023
surv_sat_f_cost['Time period'] = 2023
surv_sat_f_cost.drop(surv_sat_f_cost.columns[2], axis=1, inplace=True)

# Remove anything between square brackets in Responses
surv_sat_f_cost['Response'] = surv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_cost['Response'] = pd.Categorical(surv_sat_f_cost['Response'], categories=surv_sat_f_cost['Response'].unique(), ordered=True)
converttojson(surv_sat_f_cost,'survey_wellbeing_cost')
surv_sat_f_cost

# Last Year Cost of Living Resident survey ----------------------------------------------------------
q032 = lydataframes['lastyearresident-survey-results---financial']
q4ay22_1 = lydataframes['lastyearresident-survey-results---payforfood']
q4ay22_2 = lydataframes['lastyearresident-survey-results---payforenergy']
q4ay22_3 = lydataframes['lastyearresident-survey-results---payforrent']

# Join all DataFrames
lysurv_sat_f_cost = pd.concat([q032, q4ay22_1, q4ay22_2, q4ay22_3], ignore_index=True)

# Add a 'Year' column with value 2022
lysurv_sat_f_cost['Time period'] = 2022
lysurv_sat_f_cost

# Remove anything between square brackets in Responses
lysurv_sat_f_cost['Response'] = lysurv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lysurv_sat_f_cost['Response'] = pd.Categorical(lysurv_sat_f_cost['Response'], categories=lysurv_sat_f_cost['Response'].unique(), ordered=True)
converttojson(lysurv_sat_f_cost,'lastyearsurvey_wellbeing_cost')
lysurv_sat_f_cost

DataFrame 'survey_wellbeing_cost' saved as JSON file: Clean Data/survey_wellbeing_cost.json
DataFrame 'lastyearsurvey_wellbeing_cost' saved as JSON file: Clean Data/lastyearsurvey_wellbeing_cost.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2998876437.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_cost['Response'] = surv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\2998876437.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  lysurv_sat_f_cost['Response'] = lysurv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')


,Response,Percent,Question,Time period
0,Got worse,49.3,"Compared with this time last year, do you thin...",2022
1,Stayed the same,28.7,"Compared with this time last year, do you thin...",2022
2,Improved,19.9,"Compared with this time last year, do you thin...",2022
3,Very difficult,6.8,"Over the next year, how easy or difficult do y...",2022
4,Difficult,9.6,"Over the next year, how easy or difficult do y...",2022
5,Fairly difficult,10.8,"Over the next year, how easy or difficult do y...",2022
6,Fairly easy,31.2,"Over the next year, how easy or difficult do y...",2022
7,Easy,20.0,"Over the next year, how easy or difficult do y...",2022
8,Very easy,17.5,"Over the next year, how easy or difficult do y...",2022
9,Very difficult,18.0,"Over the next year, how easy or difficult do y...",2022


In [239]:
# Health Wellbeing Resident survey ----------------------------------------------------------
q029 = dataframes['wellbeing---resident-survey-worthwhile']
q030 = dataframes['wellbeing---resident-survey-anxiety']

# Add a space to each row of the Response column in q030
q030['Response'] = q030['Response'] + " "

# Join all DataFrames
surv_sat_f_health = pd.concat([q029, q030], ignore_index=True)
surv_sat_f_health.drop(surv_sat_f_health.columns[2], axis=1, inplace=True)

# Add a 'Year' column with value 2023
surv_sat_f_health['Time period'] = 2023

# Remove anything between square brackets in Responses
surv_sat_f_health['Response'] = surv_sat_f_health['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_health['Response'] = pd.Categorical(surv_sat_f_health['Response'], categories=surv_sat_f_health['Response'].unique(), ordered=True)
converttojson(surv_sat_f_health,'survey_wellbeing_health')
surv_sat_f_health

# Last year Health Wellbeing Resident survey ----------------------------------------------------------
q029 = lydataframes['lastyearwellbeing---resident-survey-worthwhile']
q030 = lydataframes['lastyearwellbeing---resident-survey-anxiety']

# Add a space to each row of the Response column in q030
q030['Response'] = q030['Response'] + " "

# Join all DataFrames
lastyearsurv_sat_f_health = pd.concat([q029, q030], ignore_index=True)

# Add a 'Year' column with value 2022
lastyearsurv_sat_f_health['Time period'] = 2022

# Remove anything between square brackets in Responses
lastyearsurv_sat_f_health['Response'] = lastyearsurv_sat_f_health['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lastyearsurv_sat_f_health['Response'] = pd.Categorical(lastyearsurv_sat_f_health['Response'], categories=lastyearsurv_sat_f_health['Response'].unique(), ordered=True)

converttojson(lastyearsurv_sat_f_health,'lastyearsurvey_wellbeing_health')
lastyearsurv_sat_f_health

DataFrame 'survey_wellbeing_health' saved as JSON file: Clean Data/survey_wellbeing_health.json
DataFrame 'lastyearsurvey_wellbeing_health' saved as JSON file: Clean Data/lastyearsurvey_wellbeing_health.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\840961249.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_health['Response'] = surv_sat_f_health['Response'].str.replace(r'\[.*\]', '')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\840961249.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  lastyearsurv_sat_f_health['Response'] = lastyearsurv_sat_f_health['Response'].str.replace(r'\[.*\]', '')


,Response,Percent,Question,Time period
0,10 - Completely worthwhile,25.2,"Overall, to what extent do you feel the things...",2022
1,9,11.3,"Overall, to what extent do you feel the things...",2022
2,8,27.6,"Overall, to what extent do you feel the things...",2022
3,7,16.5,"Overall, to what extent do you feel the things...",2022
4,6,5.9,"Overall, to what extent do you feel the things...",2022
5,5,5.6,"Overall, to what extent do you feel the things...",2022
6,4,1.3,"Overall, to what extent do you feel the things...",2022
7,3,1.3,"Overall, to what extent do you feel the things...",2022
8,2,0.6,"Overall, to what extent do you feel the things...",2022
9,1,0.3,"Overall, to what extent do you feel the things...",2022


In [240]:
#Current year Council and Overall / Improving the Area survey results
q01 = dataframes['resident-survey-results---council-satisfaction-of-life']
q02 = dataframes['resident-survey-results---council-runs-things']
q03 = dataframes['resident-survey-results---council-value']
q04 = dataframes['resident-survey-results---council-informed']
q05 = dataframes['resident-survey-results---council-influence-decisions']
q06 = dataframes['resident-survey-results---improving-the-area'].iloc[:10]
q07 = dataframes['resident-survey-results---top3improve'].iloc[:10]

# Assuming q07 is a pandas DataFrame and Response is one of its columns
q07['Response'] = q07['Response'] + " "

q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
q07['Response'] = q07['Response'].str.replace("The level of crime", "The level of crime ", regex=False)
surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05], ignore_index=True)
surv_sat_f_general.drop(surv_sat_f_general.columns[2], axis=1, inplace=True)
surv_sat_f_general['Year']=2023

surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'survey_data-overallandcouncil')

surv_sat_f_general = pd.concat([q06, q07], ignore_index=True)
surv_sat_f_general.drop(surv_sat_f_general.columns[2], axis=1, inplace=True)
surv_sat_f_general['Year']=2023

surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'survey_data-improvingthearea')

#Last year Community - Council and Overall / Improving the Area survey results -------------------
q01 = lydataframes['lastyearresident-survey-results---council-satisfaction-of-life']
q02 = lydataframes['lastyearresident-survey-results---council-runs-things']
q03 = lydataframes['lastyearresident-survey-results---council-value']
q04 = lydataframes['lastyearresident-survey-results---council-informed']
q05 = lydataframes['lastyearresident-survey-results---council-influence-decisions']
q06 = lydataframes['lastyearresident-survey-results---improving-the-area'].iloc[:10]
q07 = lydataframes['lastyearresident-survey-results---top3improve'].iloc[:10]

# Assuming q07 is a pandas DataFrame and Response is one of its columns
q07['Response'] = q07['Response'] + " "

q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
q07['Response'] = q07['Response'].str.replace("The level of crime", "The level of crime ", regex=False)
surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05], ignore_index=True)
surv_sat_f_general['Year']=2022
surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'lastyearsurvey_data-overallandcouncil')

surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05, q06, q07], ignore_index=True)
surv_sat_f_general['Year']=2022
surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'lastyearsurvey_data-improvingthearea')
surv_sat_f_general

DataFrame 'survey_data-overallandcouncil' saved as JSON file: Clean Data/survey_data-overallandcouncil.json
DataFrame 'survey_data-improvingthearea' saved as JSON file: Clean Data/survey_data-improvingthearea.json
DataFrame 'lastyearsurvey_data-overallandcouncil' saved as JSON file: Clean Data/lastyearsurvey_data-overallandcouncil.json
DataFrame 'lastyearsurvey_data-improvingthearea' saved as JSON file: Clean Data/lastyearsurvey_data-improvingthearea.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3950047793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'] + " "
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3950047793.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19028\3950047793.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Response,Percent,Question,Year
0,Very satisfied,22.6,"Overall, how satisfied or dissatisfied are you...",2022
1,Fairly satisfied,49.3,"Overall, how satisfied or dissatisfied are you...",2022
2,Neither satisfied nor dissatisfied,12.6,"Overall, how satisfied or dissatisfied are you...",2022
3,Fairly dissatisfied,9.6,"Overall, how satisfied or dissatisfied are you...",2022
4,Very dissatisfied,5.9,"Overall, how satisfied or dissatisfied are you...",2022
5,Very satisfied,8.8,"Overall, how satisfied or dissatisfied are you...",2022
6,Fairly satisfied,40.2,"Overall, how satisfied or dissatisfied are you...",2022
7,Neither satisfied nor dissatisfied,19.0,"Overall, how satisfied or dissatisfied are you...",2022
8,Fairly dissatisfied,18.7,"Overall, how satisfied or dissatisfied are you...",2022
9,Very dissatisfied,13.2,"Overall, how satisfied or dissatisfied are you...",2022


In [241]:
# Current year Local area survey results
q08 = dataframes['resident-survey-results---local-area-diffbackgrounds'] 
q091 = dataframes['resident-survey-results---need-help']
q092 = dataframes['resident-survey-results---work-together']

# Concatenate (join) all dataframes
surv_sat_f_local = pd.concat([q08, q091, q092])
surv_sat_f_local.drop(surv_sat_f_local.columns[2], axis=1, inplace=True)

# Remove prefix from Question column
surv_sat_f_local['Question'] = surv_sat_f_local['Question'].str.replace("Please say how strongly you agree or disagree with each of the following statements - ", "")

# Create an ordered factor for the Response column
Response_unique_loc = surv_sat_f_local['Response'].unique()
surv_sat_f_local['Response'] = pd.Categorical(surv_sat_f_local['Response'], categories=Response_unique_loc, ordered=True)
converttojson(surv_sat_f_local, 'survey_data-localarea')

# Last year Local area survey results------------------------------------------------------
q08 = lydataframes['lastyearresident-survey-results---local-area-diffbackgrounds'] 
q091 = lydataframes['lastyearresident-survey-results---need-help']
q092 = lydataframes['lastyearresident-survey-results---work-together']

# Concatenate (join) all dataframes
surv_sat_f_local = pd.concat([q08, q091, q092])

# Remove prefix from Question column
surv_sat_f_local['Question'] = surv_sat_f_local['Question'].str.replace("Please say how strongly you agree or disagree with each of the following statements - ", "")

# Create an ordered factor for the Response column
Response_unique_loc = surv_sat_f_local['Response'].unique()
surv_sat_f_local['Response'] = pd.Categorical(surv_sat_f_local['Response'], categories=Response_unique_loc, ordered=True)
converttojson(surv_sat_f_local, 'lastyearsurvey_data-localarea')
surv_sat_f_local

DataFrame 'survey_data-localarea' saved as JSON file: Clean Data/survey_data-localarea.json
DataFrame 'lastyearsurvey_data-localarea' saved as JSON file: Clean Data/lastyearsurvey_data-localarea.json


,Response,Percent,Question
8,Definitely agree,27.5,To what extent do you agree or disagree that y...
11,Tend to agree,54.5,To what extent do you agree or disagree that y...
14,Tend to disagree,8.9,To what extent do you agree or disagree that y...
17,Definitely disagree,4.6,To what extent do you agree or disagree that y...
8,Strongly agree,20.2,The friendships and associations I have with o...
11,Agree,37.6,The friendships and associations I have with o...
14,Neither agree nor disagree,26.1,The friendships and associations I have with o...
17,Disagree,8.4,The friendships and associations I have with o...
20,Strongly disagree,5.7,The friendships and associations I have with o...
8,Strongly agree,12.7,If I needed advice about something I could go ...
